In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [2]:
df_train = pd.read_csv("hands_SIBI_training.csv", header=0)

df_train.head()

,class_type,thumb_fingerX,thumb_fingerY,index_fingerX,index_fingerY,middle_fingerX,middle_fingerY,ring_fingerX,ring_fingerY,pinky_fingerX,pinky_fingerY
0,A,701.420128,318.919480,915.017724,875.987053,1033.827782,1007.422090,1137.799859,1162.527680,1299.975634,1230.812311
1,A,738.009572,546.695232,896.438837,982.484698,1015.012383,1089.546204,1108.672738,1195.127249,1244.127035,1195.254564
2,A,649.540782,382.200122,865.426779,934.989333,987.588286,1071.101904,1085.783601,1227.977633,1242.133617,1304.133892
3,A,684.587240,395.731837,797.839999,821.599484,967.001498,907.189965,1098.765850,1031.728745,1269.232035,1040.464640
4,A,678.897142,390.343607,773.604393,815.742195,949.358344,898.147583,1081.417561,1023.652792,1254.388928,1041.034222


In [3]:
df_test = pd.read_csv("hands_SIBI_validation.csv", header=0)

df_test.head()

,class_type,thumb_fingerX,thumb_fingerY,index_fingerX,index_fingerY,middle_fingerX,middle_fingerY,ring_fingerX,ring_fingerY,pinky_fingerX,pinky_fingerY
0,A,482.530147,663.602710,618.714333,1108.978033,742.493629,1213.959575,836.013734,1319.635391,970.602274,1320.696831
1,A,527.143121,691.565394,665.788949,1149.155378,780.918658,1242.375493,872.791350,1346.536756,1007.527590,1351.095319
2,B,869.509935,1152.306914,693.085134,513.347089,851.270974,442.188203,1007.714987,524.867892,1139.157057,763.286769
3,B,864.763677,1156.251431,684.375286,496.557355,844.287634,432.770491,994.463921,515.173078,1129.500747,759.229779
4,C,241.097152,942.816019,407.389641,442.306489,375.943720,413.123608,413.292289,486.609697,547.516942,640.499532


In [4]:
df_train["class_type"] = pd.Categorical(df_train["class_type"])
df_train["class_type"] = df_train.class_type.cat.codes

df_test["class_type"] = pd.Categorical(df_test["class_type"])
df_test["class_type"] = df_test.class_type.cat.codes

In [5]:
y_train = df_train.pop("class_type")
x_train = df_train.copy()

y_test = df_test.pop("class_type")
x_test = df_test.copy()

x_train = np.array(x_train)
x_test = np.array(x_test)

In [6]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [7]:
print(x_train[0])
print(x_test[0])

[[ 701.42 ]
 [ 318.919]
 [ 915.018]
 [ 875.987]
 [1033.828]
 [1007.422]
 [1137.8  ]
 [1162.528]
 [1299.976]
 [1230.812]]
[[ 482.53 ]
 [ 663.603]
 [ 618.714]
 [1108.978]
 [ 742.494]
 [1213.96 ]
 [ 836.014]
 [1319.635]
 [ 970.602]
 [1320.697]]


In [8]:
num_classes = 26

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(64, kernel_size=3, strides=1, activation="relu", input_shape=x_train.shape[1:3]),
  tf.keras.layers.MaxPooling1D(pool_size=2),
  tf.keras.layers.Conv1D(128, kernel_size=3, strides=1, activation="relu"),
  tf.keras.layers.MaxPooling1D(pool_size=2), 
  # Flatten the results to feed into a DNN
  tf.keras.layers.Flatten(), 
  # 512 neuron hidden layer
  tf.keras.layers.Dense(512, activation='relu'), 
  tf.keras.layers.Dense(num_classes, activation='softmax')])

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 8, 64)             256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 4, 64)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2, 128)            24704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 128)            0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               66048     
_________________________________________________________________
dense_1 (Dense)              (None, 26)                1

In [10]:
model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/100
7/7 [==============================] - 1s 115ms/step - loss: 98.0709 - accuracy: 0.0554 - val_loss: 73.2984 - val_accuracy: 0.1000
Epoch 2/100
7/7 [==============================] - 0s 17ms/step - loss: 43.2369 - accuracy: 0.1178 - val_loss: 34.2721 - val_accuracy: 0.1000
Epoch 3/100
7/7 [==============================] - 0s 17ms/step - loss: 19.7580 - accuracy: 0.1848 - val_loss: 8.8778 - val_accuracy: 0.2200
Epoch 4/100
7/7 [==============================] - 0s 18ms/step - loss: 4.6457 - accuracy: 0.3857 - val_loss: 3.4493 - val_accuracy: 0.5000
Epoch 5/100
7/7 [==============================] - 0s 18ms/step - loss: 3.0048 - accuracy: 0.4688 - val_loss: 2.3344 - val_accuracy: 0.5400
Epoch 6/100
7/7 [==============================] - 0s 32ms/step - loss: 2.3985 - accuracy: 0.5381 - val_loss: 2.7622 - val_accuracy: 0.5200
Epoch 7/100
7/7 [==============================] - 0s 18ms/step - loss: 1.7751 - accuracy: 0.6236 - val_loss: 1.2889 - val_accuracy: 0.6000
Epoch 8/100
7/

In [11]:
input_test = [[[482.5301468372345],[663.6027097702026],[618.7143325805664],[1108.978033065796],[742.4936294555664],[1213.9595746994019],[836.0137343406677],[1319.6353912353516],[970.60227394104],[1320.6968307495117]]]
input_test = np.array(input_test)
input = np.reshape(input_test, (input_test.shape[0], input_test.shape[1], 1))
print(input_test.shape)
print(input_test)

(1, 10, 1)
[[[ 482.53 ]
  [ 663.603]
  [ 618.714]
  [1108.978]
  [ 742.494]
  [1213.96 ]
  [ 836.014]
  [1319.635]
  [ 970.602]
  [1320.697]]]


In [14]:
print(model.predict(input_test))
print(model.predict_classes(input_test))

[[0.805 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.001 0.    0.    0.    0.
  0.194 0.   ]]
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[0]


In [15]:
save_model = "model_SIBI.h5"
model.save(save_model)